# Python para el análisis de datos -  UNAV
---

# Streamlit

## Introducción a Streamlit<a name="streamlit"></a> 
[Volver al índice](#indice)

[Streamlit](https://streamlit.io/) es una libreria que ha ido ganando peso ultimamente (hay otras que hacen algo parecido), porque permite que gente con cero experiencia en lenguages front-end como HTML, CSS, Javascript, puedan mostrar dashboards interactivos de forma muy rapida y sencilla.

Además, permiten hostear esos dashboard en sus servidores de forma gratuita (hasta cierto punto, claro) con lo que es muy facil compartirlo o distribuirlo con quien queramos. Por otro lado cuenta con una comunidad muy activa que ayuda mucho es y es participativa.

Dada la propia naturaleza de Streamlit, cuyo objetivo es generar una aplicación web funcional rápidamente, es dificil (pero no imposible) ejecutarlo en un Notebook, así que vamos a ver un ejemplo utilizando un proyecto aparte.








### Instalacion streamlit<a name="streamlit"></a> 
[Volver al índice](#indice)


Streamlit se instala como cualquier otro paquete en Python. Asumimos que teneís instalado ya pip por lo menos, aunque se recomienda pipenv o poetry.
Además, Streamlit no funciona con Python 3 todavía, por tanto, o lo ejecutáis en un entorno virtual o lo ejecutáis específicamente con python 2 o no tenéis instalado python 3.

`` pip install streamlit``

o

`` conda install streamlit``

Streamlit CLI deberia estar listo, y podemos probar con la pagina de bienvenida, que nos abrira el navegador por defecto con la aplicación de sreamlit

`` streamlit hello``

Para ejecutar streamlit u os colocáis donde esté la instalación o añadís esa instalación al path.


Vamos a crear un script en Python para ver como funciona Streamlit, crear un fichero .py, es recomendable que dividaís la pantalla en 2 de modo que en un lado tengaís el editor con el que esteís modificando el script y en el otro el navegador. Supongamos que nuestro fichero se llama **streamlit_tutorial.py**.

Lo primero que haremos sera importar los modulos que necesitamos.

In [4]:
# pip install streamlit

import pandas as pd
import streamlit as st
import plotly.express as px

Ahora podemos cargar los datos, en este caso se trata de un dataset con datos de las propiedades en Nueva York de Airbnb, de la pagina insideairbnb. Usamos el decorator st.catche para poder cachear la llamada que carga los datos, con lo que no se ejecuta cada vez que ejecutamos el script.

In [ ]:
@st.cache
def get_data():
    url = "http://data.insideairbnb.com/the-netherlands/north-holland/amsterdam/2022-09-07/visualisations/listings.csv"
    return pd.read_csv(url)
df = get_data()

Veamos como mostrar texto y el DataFrame, que por cierto se puede ordenar por columnas!

In [ ]:
st.title("Streamlit tutorial: Usando Streamlit")
st.markdown("Este tutorial muestra como usar Streamlit para mostrar datos de Airbnb.")
st.header("Airbnb")
st.markdown("Datos de Amsterdam 2022")

st.dataframe(df.head())

Streamlit hace muy facil mostrar datos geo, solo hace falta que el dataset tenga columnas lat, long. Veamos como:

In [ ]:
st.header("Localizacion de los Airbnb mas caros en Amsterdam")
st.subheader("En un mapa")
st.markdown("El mapa siguiente muestra los apartamentos mas caros de 800 dolares por noche.")
st.map(df.query("price>=800")[["latitude", "longitude"]].dropna(how="any"))

st.subheader("En una table")
st.markdown("Lista de las 10 propiedades mas caras.")

Te permite imprimir en cualquier momento el dataframe a tu gusto:

In [ ]:
st.write(df.query("price>=800").sort_values("price", ascending=False).head(10))

Streamlit permite especificar que subconjunto de columnas queremos usar

In [ ]:
st.subheader("Seleccionando un subconjunto de columnas del dataframe")
st.write(
    f"De las {df.shape[1]} columnas, puede que queramos solo algunas, Streamlit tiene un widget para eso.")
defaultcols = ["name", "host_name", "neighbourhood", "room_type", "price"]
cols = st.multiselect("Columns", df.columns.tolist(), default=defaultcols)
st.dataframe(df[cols].head(5))

Tambien podemos crear tablas estaticas, pero son menos interactivas que el dataframe

In [ ]:
st.header("Precio medio por tipo de habitacion")
st.write("Podemos crear una tabla estatica, pero no permite interactuar con ella como con el dataframe")
st.table(df.groupby("room_type").price.mean().reset_index() \
         .round(2).sort_values("price", ascending=False) \
         .assign(avg_price=lambda x: x.pop("price").apply(lambda y: "%.2f" % y)))

Podemos mostrar tambien resultas de hacer queries como JSON, lo cual puede ser muy interesante, veamos algunas propiedades de los host con mas propiedades

In [ ]:
st.header("Que host tiene mas propiedades listadas")
listingcounts = df.host_id.value_counts()
top_host_1 = df.query('host_id==@listingcounts.index[0]')
top_host_2 = df.query('host_id==@listingcounts.index[1]')
st.write(
    f"""**{top_host_1.iloc[0].host_name}** es el host que tiene mas propiedades listadas, en total {listingcounts.iloc[0]}.
**{top_host_2.iloc[1].host_name}** es el segundo propietario con  {listingcounts.iloc[1]} propiedades listadas. A continuacion
se muestran listados aleatorios de las propiedades de cada uno, usando JSON[`st.json`](https://streamlit.io/docs/api.html#streamlit.json).""")
st.json({top_host_1.iloc[0].host_name: top_host_1 \
    [["name", "neighbourhood", "room_type", "minimum_nights", "price"]] \
        .sample(2, random_state=4).to_dict(orient="records"),
         top_host_2.iloc[0].host_name: top_host_2 \
             [["name", "neighbourhood", "room_type", "minimum_nights", "price"]] \
        .sample(2, random_state=4).to_dict(orient="records")})

Una de las principales caracteristicas de Streamlit es que permite interactuar, veamos como mostrar propiedades en base a un rango de precio, que mostraremos en una barra lateral.

In [ ]:
st.header("Cual es la distribucion del precio?")
st.write("""Selecciona un rango de precio de la barra lateral para actualizar el historigrama Plotly chart
[`st.plotly_chart`](https://streamlit.io/docs/api.html#streamlit.plotly_chart).""")
values = st.sidebar.slider("Rango de precio", float(df.price.min()), float(df.price.clip(upper=1000.).max()),
                           (50., 300.))
f = px.histogram(df.query(f"price.between{values}"), x="price", nbins=15, title="Distribucion del precio")
f.update_xaxes(title="Precio")
f.update_yaxes(title="# de listados")
st.plotly_chart(f)

Tambien podemos filtar en base al numero de reviews minimo y maximo

In [ ]:
st.header("Propiedades en base a las reviews")
minimum = st.sidebar.number_input("Minimo", min_value=0)
maximum = st.sidebar.number_input("Maximo", min_value=0, value=5)
if minimum > maximum:
    st.error("Por favor introducir un rango valido")
else:
    df.query("@minimum<=number_of_reviews<=@maximum").sort_values("number_of_reviews", ascending=False)\
        .head(50)[["name", "number_of_reviews", "neighbourhood", "host_name", "room_type", "price"]]

Podemos tambien cargar imagenes...

In [ ]:
st.header("Imagenes")
pics = {
    "Cat": "https://cdn.pixabay.com/photo/2016/09/24/22/20/cat-1692702_960_720.jpg",
    "Puppy": "https://cdn.pixabay.com/photo/2019/03/15/19/19/puppy-4057786_960_720.jpg",
    "Sci-fi city": "https://storage.needpix.com/rsynced_images/science-fiction-2971848_1280.jpg"
}
pic = st.selectbox("Seleccion de imagenes", list(pics.keys()), 0)
st.image(pics[pic], use_column_width=True, caption=pics[pic])

Pero veamos juntos la documentación:
https://docs.streamlit.io/library/api-reference